In [1]:
%use serialization, intellij-platform

IntelliJ Platform integration is loaded

In [3]:
import java.io.File
import java.nio.file.Path

fun String.substitutePlaceholders(config: EduConfig): String {
    // сортируем по убыванию offset, чтобы замены не сдвигали последующие
    return config.placeholders
        .sortedByDescending { it.offset }
        .fold(this) { acc, ph ->
            acc.replaceRange(ph.offset, ph.offset + ph.length, ph.placeholder_text)
        }
}

@Serializable
data class LessonsList(
    val content: List<String>
)

@Serializable
data class Placeholder(
    val offset: Int,
    val length: Int,
    val placeholder_text: String
)

@Serializable
data class EduConfig(
    val name: String,
    val visible: Boolean,
    val placeholders: List<Placeholder> = emptyList()
)

@Serializable
data class TaskInfo(
    val type: String,
    val files: List<EduConfig>,
    val feedback_link: String
)

data class Lesson(val taskInfo: TaskInfo, val folderName: Path, val description: String)


In [4]:
USE {
    dependencies {
        implementation("com.charleskorn.kaml:kaml-jvm:0.94.0")
    }
}

loadBundledPlugins("com.intellij.notebooks.core", "intellij.jupyter")

In [5]:
import java.nio.file.Paths
val dir = Paths.get("/Users/Roman.Belov/Documents/JB/kotlin-koans-edu/Introduction")

In [6]:
import com.charleskorn.kaml.Yaml

val lessons = Yaml
    .default.decodeFromString<LessonsList>(dir.resolve("lesson-info.yaml").toFile().readText())
    .content.map {
        Lesson(
            Yaml.default.decodeFromString<TaskInfo>(dir.resolve("${it}/task-info.yaml").toFile().readText()),
            dir.resolve("${it}"),
            dir.resolve("${it}/task.md").toFile().readText()
        )
    }

In [7]:
import com.intellij.openapi.util.io.toNioPath
import com.intellij.openapi.util.io.toNioPathOrNull
import com.intellij.openapi.vfs.VirtualFileManager
import okio.Path.Companion.toPath
import com.intellij.jupyter.core.jupyter.helper.jupyterNotebookOrNull
import com.intellij.openapi.editor.impl.EditorImpl
import com.intellij.openapi.fileEditor.TextEditor

val currentNotebook = currentEditor()?.file?.jupyterNotebookOrNull!!
val editor = (currentEditor() as? TextEditor)?.editor as EditorImpl
currentNotebook

com.intellij.jupyter.core.jupyter.nbformat.JupyterNotebookBase@5ed45778

In [8]:
import com.intellij.jupyter.core.core.impl.actions.NotebookCellLinesDocumentUtils.insertCells
import com.intellij.jupyter.core.core.impl.actions.NotebookCellLinesEditorUtils.withIgnoreStandardEditorPositionKeeper
import com.intellij.notebooks.visualization.NotebookCellLines
import com.intellij.openapi.command.WriteCommandAction

fun addNewCell(
    cellText: String,
    targetOrdinal: Int,
    onInsert: (NotebookCellLines.Interval) -> Unit,
) {
    val project = currentProject()
    val cellLines = NotebookCellLines.get(editor)
    WriteCommandAction.writeCommandAction(project).withName("Inserting cell").run<RuntimeException> {
        val insertedCells = editor.withIgnoreStandardEditorPositionKeeper {
            editor.document.insertCells(cellLines, cellText, targetOrdinal)
        }
        insertedCells.firstOrNull()?.let { cellInterval ->
            onInsert(cellInterval)
        }
    }
}

fun addMarkDown(md: String) = addNewCell("""
$mdHeader
${md}
""".trimIndent(), currentNotebook.cellsCount() - 1, {})

fun addCode(md: String) = addNewCell("""
$codeHeader
${md}
""".trimIndent(), currentNotebook.cellsCount() - 1, {})

In [9]:
fun startLesson(lessonNumber: Int = 0) {
    addMarkDown(lessons[lessonNumber].description)
    addCode(
        lessons[lessonNumber].taskInfo.files
            .first { it.name.contains("Task.kt") }
            .let {
                lessons[lessonNumber].folderName.resolve(it.name)
                    .toFile().readText().substitutePlaceholders(it)
            })
    notebook.afterCellExecutionsProcessor.unregisterAll()
    USE {
        afterCellExecution { _, _ ->
//            println("After cell execution: ${resultField.name} = ${resultField.value}")
            // Variable x will be visible inside the notebook
            execute(
                """EXECUTE(lessons[$lessonNumber].folderName.resolve(lessons[$lessonNumber].taskInfo.files.filterNot { it.visible }.first().name).toFile().readText())
                """.trimEnd()
            )
            val regex = Regex("""class\s+(\w+)\s*""")
            val test = lessons[lessonNumber].folderName.resolve(lessons[lessonNumber].taskInfo.files.filterNot { it.visible }.first().name).toFile().readText()
            val match = regex.find(test)!!
            if (execute(
                """
                import org.junit.runner.Description
                import org.junit.runner.Runner
                import org.junit.runner.Request
                import org.junit.runner.JUnitCore
                import org.junit.runner.notification.Failure
                import org.junit.runner.notification.RunNotifier
                val instance = ${match.groupValues[1]}()
                val request = Request.runner(InstanceRunner(instance))
                val result = JUnitCore().run(request)

                result.failures.forEach { println("Test failed: ${'$'}{it.message}") }
                if (result.wasSuccessful()){
                  println("You rock!")
                } else
                  println("Try again!")
                result.wasSuccessful()
                """
            ).value == true){
                execute("""startLesson(${lessonNumber+1})""")
            }
        }
    }
}

In [10]:
notebook.afterCellExecutionsProcessor.unregisterAll()

In [11]:
startLesson()